# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2023


> *Este notebook apresenta aspectos da programação com CUDA usando a plataforma Google Colab. A ideia é usar o mecanismo de documentação e programação, aproveitando-se do fato que esta plataforma permite associar GPUs à execução dos códigos*.




https://www.geeksforgeeks.org/how-to-run-cuda-c-c-on-jupyter-notebook-in-google-colaboratory/


https://github.com/eegkno/CUDA_by_practice

# Sobre GPUs e CUDA

Do ponto de vista do hardware, as GPUs são aglomerados de processadores organizados de forma hierárquica, tanto na interligação dos processadores quanto no acesso a módulos de memória presentes no adaptador.

GPUs foram projetadas inicialmente para o paralelismo de dados, sendo que todos os processadores executam o mesmo conjunto de instruções ao mesmo tempo, comumente sobre partes distintas dos dados.

Além de servirem para a geração eficiente de informações que são apresentadas em dispositivos de vídeo, há mecanismos de programação que conseguem utilizar os processadores de GPUs para **operações de propósito geral**, caracterizando-as como GP-GPUs (*General Purpose GPUs*).

CUDA é uma arquitetura de software e hardware que permite que [GPUs NVIDIA](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#cuda-a-general-purpose-parallel-computing-platform-and-programming-model) executem programas escritos em C, C++, Fortran, OpenCL, DirectCompute, e outras linguagens. Essa arquitetura foi pioneira em prover um [modelo de programação](https://docs.nvidia.com/cuda/cuda-c-programming-guide/#cuda-a-general-purpose-parallel-computing-platform-and-programming-model) de propósito geral usando GPUs.

Posteriormente, um padrão aberto para a programação com GPUs, chamado **OpenCL**, passou a ser desenvolvido por um conjunto de empresas, incluindo NVIDIA. Ambos os modelos de programação, CUDA e OpenCL, têm abordagens semelhantes.

Mais recentemente, OpenAcc e OpenMP também incorporaram mecanismos para usar GPUs no processamento de tarefas.

O modelo de programação para GP-GPUs  é caracterizado como **SIMT**: *Single Instruction Multiple Thread*, uma vez que a mesma *thread*, comumente formada por pequenos conjuntos de instruções, é executada em todos os processadores da GPU ao mesmo tempo.

https://docs.nvidia.com/cuda/cuda-c-programming-guide/_images/gpu-computing-applications.png

# Programando com CUDA no Colab

Para usar o suporte à programação com CUDA e GPUs no Colab, é preciso, inicialmente, fazer a configuração do Notebook para incluir uma GPU.

Assim, no menu superior, selecione **Edit** -> **Notebook Settings** (ou **Runtime** -> **Change runtime type**) e na opção **Hardware Accelerator**, habilitar uma **GPU**.

Tendo feito isso, uma estratégia para execução dos códigos é instalar um [plugin](https://github.com/andreinechaev/nvcc4jupyter) que permite executar diretamente programas **CUDA** nas células de código.

Também é possível compilar manualmente os códigos, usando o compilador C NVIDIA, [nvcc](https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html), como veremos nos exemplos.

In [ ]:
# importa macro %%cu , que permite compilar e executar diretamente o código de uma célula do notebook.
! pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

# carrega plugin do material importado acima
%load_ext nvcc_plugin

# exibindo informações sobre as versões de compiladores disponíveis
! echo && nvcc --version
! echo && gcc --version
! echo && g++ --version

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7ugg5d8p
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7ugg5d8p
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=a012542cc7c9b3132116ff2327521cf63ba98765b068a21557605833553323c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-u9hkd0kv/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0

gcc (Ubuntu 11.4.0-1ubuntu1~22

## Nota sobre os modelos de GPU e compilação:

Considerando que a versão CUDA desta plataforma foi recentemente atualizada para a versão 11, que não gera código nativo para as GPUs k80, que podem ser alocadas às sessões do Colab, pode ser preciso incluir um parâmetro a mais na linha de compilação, especificando a arquitetura alvo, caso a GPU atribuíd ao Colab seja a K80.

ex:   
```
nvcc prog.cu -o prog    -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37
```

# Obtendo detalhes da GPU disponível

Tendo associado uma GPU à sessão, é possível obter informações sobre sua configuração.

O comando [nvidia-smi](https://developer.nvidia.com/nvidia-system-management-interface) apresenta informações sobre a GPU disponibilizada pelo Google Colaboratory nesta sessão do Colab.


In [ ]:
! nvidia-smi

Sun Jan  7 18:19:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Informações sobre a GPU também podem ser obtidas via programa, através da função da API CUDA [cudagetDeviceProperties](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g1bf9d625a931d657e08db2b4391170f0), que retorna uma estrutura do tipo [cudaDeviceProp](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html#structcudaDeviceProp).

Algumas dessas informações são ilustradas a seguir.

In [ ]:
%%cu
// esta macro salva o arquivo desta célula, compila seu código com nvcc e executa o código gerado

#include <stdio.h>

int main(void)
{
  int deviceCount = 0;
  cudaGetDeviceCount(&deviceCount);

  if(deviceCount == 0) {
    printf("Não há dispositivos compatíveis com CUDA disponíveis no sistema.\n");
    return 0;
  }
  // devine um dispositivo para execução (necessário apenas se houver mais de 1 GPUs)
	cudaSetDevice(0);
	cudaDeviceProp prop;
	cudaGetDeviceProperties(&prop,0);

 	printf("Modelo do Device: %s\n",prop.name);
  printf("Número de SMs: %d\n",prop.multiProcessorCount);
  printf("Número de Regs por SM: %d K\n",prop.regsPerMultiprocessor >> 10);
  printf("Número de Regs por Bloco: %d K\n",prop.regsPerBlock  >> 10);
  printf("Memória compartilhada por SM: %d KB\n", (int)prop.sharedMemPerMultiprocessor >> 10);
  printf("Memória compartilhada por Bloco: %d KB\n",(int)prop.sharedMemPerBlock  >> 10);
  printf("Memória Global: %ld GB\n",(long int)prop.totalGlobalMem  >> 10  >> 10  >> 10 );
  printf("Memória Constante: %d KB\n",(int)prop.totalConstMem  >> 10);

	return 0;
}

Modelo do Device: Tesla T4
Número de SMs: 40
Número de Regs por SM: 64 K
Número de Regs por Bloco: 64 K
Memória compartilhada por SM: 64 KB
Memória compartilhada por Bloco: 48 KB
Memória Global: 14 GB
Memória Constante: 64 KB



In [ ]:
# para compilar e executar manualmente o código, seria preciso salvar o arquivo antes...
# ! nvcc dev.cu -o dev && ./dev

*As of CUDA 12.0, the [cudaInitDevice](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1gac04a5d82168676b20121ca870919419)() and [cudaSetDevice](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g159587909ffa0791bbe4b40187a4c6bb)() calls initialize the runtime and the primary context associated with the specified device. Absent these calls, the runtime will implicitly use device 0 and self-initialize as needed to process other runtime API requests.*

*The runtime creates a CUDA context for each device in the system. This context is the primary context for this device and is initialized at the first runtime function which requires an active context on this device. It is shared among all the host threads of the application. As part of this context creation, the device code is just-in-time compiled if necessary and loaded into device memory. This all happens transparently.*

*When a host thread calls [cudaDeviceReset](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1gef69dd5c6d0206c2b8d099abac61f217)(), this destroys the primary context of the device the host thread currently operates on (i.e., the current device as defined in Device Selection).*

# Criando códigos para GPUs: *kernels*

Na programação com CUDA, o fluxo principal do programa continua sendo em CPU, mas é possível transferir parte do processamento para a(s) GPU(s), bem como os dados necessários para esses processamentos.

No funcionamento combinado de CPU e GPU, usa-se o termo ***host*** para designar o sistema computacional, e ***device*** para designar a GPU. Assim, fala-se em processamento no *host* e processamento no *device* (dispositivo). Cabe ao código do *host* coordenar as transferência de dados e códigos para o *device*, bem como ativar suas execuções. Os códigos executados no *device* são chamados de ***kernels***.

De maneira geral, a programação com CUDA envolve as seguintes etapas:

* Declarar e alocar variáveis e espaços de memória para dados na RAM (*host memory*) (para as atividades executando em CPU);
* Declarar e alocar variáveis necessárias no espaço de endereçamento da GPU (*device memory*), para os processamentos em GPU;
* Inicializar dados no *host*;
* Transferir dados do *host* para o *device*;
  <br>[ ou Inicializar dados direteramente no *device*; ]
* Executar um ou mais *kernels* no *device*;
* Transferir os resultados da memória do *device* para a memória do *host*.

A declaração e a ativação de códigos para execução em GPU ocorrem dentro do próprio programa. Cada código de GPU (*kernel*) é declarado como uma função que tem o prefixo **\_\_global__**. Isso indica ao compilador para gerar código para esta função usando os recursos da arquitetura da GPU.

Variáveis declaradas dentro de uma função de *kernel* serão alocadas automaticamente na área de memória do *device*. Essa alocação pode ocorrer em registradores dos processadores da GPU ou em posições de memória, o que é feito pela própria plataforma CUDA.

Na ativação da função do *kernel*, especifica-se também a organizacão lógica das *threads* que serão utilizadas na execução deste código. Assim, o **número de instâncias** que irão executar o *kernel* especificado é determinado pelo **número de *threads*** selecionadas na ativação do *kernel*.

<!--
```
// Kernel definition
__global__ void VecAdd(float* A, float* B, float* C)
{
    int i = threadIdx.x;
    C[i] = A[i] + B[i];
}

int main()
{
    ...
    // Kernel invocation with N threads
    VecAdd<<<1, N>>>(A, B, C);
    ...
}
```
-->

<br>

O exemplo a seguir ilustra um programa que inclui a definição de um *kernel* simples e uma chamada para execuçao deste *kernel* na GPU, utilizando apenas 1 *thread*.

A ativação do código de um *kernel* em GPU é feita de forma semelhante a uma chamada de função no programa da CPU. O uso dos caracteres ""<<< , >>>"" após o nome da função serve para indicar o número e a organização lógica dos processadores da GPU. Parâmetros da função do *kernel* são especificados depois dessas informações.  


In [ ]:
# macro que compila o código CUDA desta célula e o executa
# %%cu

# macro que salva o conteúdo da célula como um arquivo
%%writefile hello.cu

#include <stdio.h>

// __global__ indica que esta função é um Kernel para execução em GPU

__global__ void helloFromGPU(){
    printf("Hello from GPU!\n");
    return;
}

void helloFromCPU(){
    printf("Hello from CPU!\n");
    return;
}

int main()
{
    // Chamada do Kernel com apenas 1 thread em 1 bloco
    helloFromGPU<<<1,1>>>();

    // Chamada da função em CPU
    helloFromCPU();

    // A função a seguir é necessária para garantir que a execução do kernel em GPU foi concluída.
    // Experimente comentar a linha a seguir e veja que o programa termina antes que a impressão
    // feita no código do kernel seja concluída
    cudaDeviceSynchronize();

    return 0;
}

Writing hello.cu


In [ ]:
# ! nvcc hello.cu -o hello -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37
! if [ ! hello -nt hello.cu ]; then nvcc hello.cu -o hello; fi
! ./hello

Hello from CPU!
Hello from GPU!


# Sincronismo CPU / GPU

Após a execução de um *kernel* ser **ativada**, o fluxo de execução de instruções em CPU continua normalmente. Assim, CPU(s) e GPU(s) trabalham em **paralelo**, de forma **assíncrona**.

Caso a conclusão do processamento em GPU seja necessária antes de realizar outra operação em CPU, é possível usar a função [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d)(\*).

Quando o processamento em GPU ocorre em ciclos, esta sincronização ocorre tipicamente antes de cada nova etapa. De maneira geral, também ocorre ao final do programa principal em CPU, para garantir que as operações em GPU foram concluídas antes de o programa ser terminado.

<br>

(\*) **cudaDeviceSynchronize** ( void ): *Blocks until the device has completed all preceding requested tasks*.

Veja mais em [Cuda Device Management](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html)






# Tratamento de erros em funções da API CUDA e na ativação do *kernel*

A maioria das funções da API CUDA, quando invocadas, tem como retorno um valor do tipo [cudaError_t](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html#group__CUDART__TYPES_1gf599e5b8b829ce7db0f5216928f6ecb6). Em caso de sucesso na execução, o valor retornado é [cudaSuccess](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html#group__CUDART__TYPES_1gg3f51e3575c2178246db0a94a430e0038e355f04607d824883b4a50662830d591) e, em caso de falha, é possível detectar a falha ocorrida usando a função [cudaGetErrorString](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html)().

```c
cudaError_t err;
err = cudaX...Y...(...);

if (err != cudaSuccess) {
  printf("Erro: %s\n", cudaGetErrorString(err));
  ...
}
```


Já na ativação de um *kernel* em GPU, não há um valor de retorno. Entretanto, erros podem acontecer e é importante identificá-los.

Isso pode ser feito usando-se a função [cudaGetLastError](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html)(), ilustrada a seguir:

```c
// Ativação de kernel
k_function <<< 1, 0>>>(); // número inválido de threads: 1 bloco com 0 threads

cudaError_t err;
err = cudaGetLastError();
if (err!= cudaSuccess) {
  printf("Erro: %s\n", cudaGetErrorString(err));
  ...
}
```

Também é possível identificar erros que ocorram **durante** a execução de um *kernel* que foi ativado com sucesso. Como a execução de um *kernel* ocorre de forma assíncrona com a execução do código em CPU, este tipo de erro é chamado de erro assíncrono.

Neste caso, a análise do valor de retorno da função [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html)() indicará a ocorrência de erros na execução de um *kernel* ativado anteriormente.

```c
  cudaError_t syncErr, asyncErr;

  k_function <<< number_of_blocks, threads_per_block>>>(...);

  // Teste de erros na ativação do kernel
  syncErr = cudaGetLastError();
  if (syncErr != cudaSuccess) {
    printf("Error: %s\n", cudaGetErrorString(syncErr));
    // termina programa (?)
  }
  ...
  // Teste de erros durante a execução do kernel
  asyncErr = cudaDeviceSynchronize();
  if (asyncErr != cudaSuccess) {
    printf("Error: %s\n", cudaGetErrorString(asyncErr));
    ...
  }
```


# Blocos e Threads

A execução de instruções em GPU com CUDA é feita no modelo **SIMT**, criando *threads* a partir de funções definidas como *kernels*.  O **número** e a **organização das *threads*** que vão executar instâncias do *kernel* simultaneamente são determinados como parâmetros na ativação do *kernel*.

* As *threads* são organizadas em **blocos**. A estrutura dos blocos é definida por uma variável do tipo ***dim3***, que possibilita arranjos em 1, 2 ou 3 dimensões.

* Além disso, é possível definir uma grade (***grid***) de blocos, que podem ser organizados em estruturas lógicas de 1, 2 ou 3 dimensões.

Assim, há 2 parâmetros que são passados na ativação de um *kernel*. O primeiro indica a estruturação do ***grid***, ou seja, quantos blocos de *threads* serão usados e como esses blocos estarão organizados. O segundo parâmetro indica como as *threads* estarão organizadas dentro de cada bloco.

```c
kernel_function <<< grid, bloco >>>
```
ou seja,
```c
kernel_function <<< #blocos, #threads_por_bloco >>>
```

Durante a execução das *threads*, há variáveis que permitem a cada uma das *threads* saber quais são as dimensões do bloco a que pertence e qual é o posicionamento desta *thread* dentro deste bloco, nas dimensões X, Y e Z.

Também é possível saber qual é a organização do *grid* e qual é o posicionamento do bloco desta *thread* em relação ao *grid*.


* ***Thread*** dentro do bloco: **threadIdx.x**, **threadIdx.y**, **threadIdx.z**

* **Bloco** dentro do *grid*: **blockIdx.x**, **blockIdx.y**, **blockIdx.z**

* **Dimensões** do bloco: **blockDim.x**, **blockDim.y**, **blockDim.z**

* **Dimensões** do grid em blocos: **gridDim.x**, **gridDim.y**, **gridDim.z**

<br>

Há 2 fatores importantes associados a essa forma de organização lógica das *threads*. Um aspecto é poder mapear o identificador da *thread* com a organização de estruturas de dados multidimensionais, como vetores, matrizes e volumes.


Outro aspecto da organização das *threads* em blocos é o mapeamento dos blocos aos elementos de processamento das GPUs.

<!--
Um ou mais blocos serão alocados para executar em multiprocessadores (**Stream Multiprocessor ou SM**). Quando um bloco é escalonado em um SM irá permanecer no SM até terminar sua execução.  **Os blocos e seus threads são executados em ordem aleatória**. A quantidade de blocos que executa simultaneamente em um SM depende da quantidade de recursos do SM e da quantidade de recursos requisitados pelo bloco (número de registradores e quantidade de memória compartilhada), além é claro do limite máximo definido para cada arquitetura de GPU.
-->

<br>

No exemplo a seguir, vê-se a ativação do *kernel* e a configuração do *grid* e das *threads* dentro dos blocos.

As variáveis ***threadIdx*** e ***blockIdx*** indicam a posição lógica de cada *thread* na estrutura.


In [ ]:
%%file index.cu

#include <cuda_runtime.h>
#include <stdio.h>

/*
  typedef struct {
    unsigned int x;
    unsigned int y;
    unsigned int z;
  } dim3;
*/


__global__
void checkIndex(void)
{
  printf("threadIdx:(%d, %d, %d) blockIdx:(%d, %d, %d) blockDim:(%d, %d, %d) "
      "gridDim:(%d, %d, %d)\n", threadIdx.x, threadIdx.y, threadIdx.z,
      blockIdx.x, blockIdx.y, blockIdx.z, blockDim.x, blockDim.y,
      blockDim.z, gridDim.x, gridDim.y, gridDim.z);
}

int
main(int argc, char **argv)
{
  dim3 grid = {1,1,3};    // organização dos blocos de threads
  dim3 block = {2,2,2};   // organização das threads em cada bloco

  checkIndex <<< grid, block >>>();

  // espera processamento na GPU ser concluído
  cudaDeviceSynchronize();

  return (0);
}


Writing index.cu


In [ ]:
# ! nvcc index.cu -o index   -Wno-deprecated-gpu-targets -gencode=arch=compute_37,code=sm_37
! nvcc index.cu -o index
! ./index

threadIdx:(0, 0, 0) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 0) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 0) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 0) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 1) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 1) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 1) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 1) blockIdx:(0, 0, 2) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 0, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 1, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1, 1, 0) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(0, 0, 1) blockIdx:(0, 0, 0) blockDim:(2, 2, 2) gridDim:(1, 1, 3)
threadIdx:(1

Examinando o resultado acima, vê-se que todas as *threads* mostram os mesmos valores para blockDim e gridDim, que indicam as **dimensões** do bloco de *threads* e do *grid* de blocos.

Já os índices das *threads* nos blocos a que pertentem e os índices dos blocos na grade variam.